In [5]:
import requests
import pandas as pd
import mysql.connector
import decimal
import pymysql

url = "https://churchofjesuschristtemples.org/maps/gps/"
filename = "temples.csv"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the content of the response (CSV file) to a local file
    with open(filename, "wb") as file:
        file.write(response.content)

 # Read the CSV file into a DataFrame
    temples = pd.read_csv(filename, header=0, encoding= 'unicode_escape')
    #print(temples.head)

else:
    print("Failed to download the file. Status code:", response.status_code)

In [6]:
# Format my dataframe to be uploaded to database
df = {'Temple': temples.Temple
    , 'Status': temples.Status
    , 'Latitude': temples.Latitude
    , 'Longitude': temples.Longitude
    , 'Address': temples.Address
    , 'City': temples.City
    , 'County': temples.County
    , 'State': temples.State
    , 'Postal_Code': temples['Postal Code']
    , 'Country': temples.Country
    , 'Phone': temples.Phone
}
df = pd.DataFrame(df)
df['Latitude'] = df['Latitude'].apply(decimal.Decimal)
df['Longitude'] = df['Longitude'].apply(decimal.Decimal)
print(df.dtypes)

Temple         object
Status         object
Latitude       object
Longitude      object
Address        object
City           object
County         object
State          object
Postal_Code    object
Country        object
Phone          object
dtype: object


In [7]:
# Connect to MySQL server
cnx = mysql.connector.connect(
    host = '23.239.4.168'
    , user = 'jkober'
    , password = '4N4CrFHevPzpdt!'
    , database = 'external_data'
)

# Create a cursor object to execute SQL queries
cursor = cnx.cursor()

# Query to drop table
drop_table_query = """
    drop table external_data.temples
"""
cursor.execute(drop_table_query)

# Query to create table
create_table_query = """
    create table external_data.temples (
        Temple varchar(50)
        , Status varchar(50)
        , Latitude float
        , Longitude float
        , Address varchar(200)
        , City varchar(50)
        , County varchar(50)
        , State varchar(50)
        , Postal_Code varchar(50)
        , Country varchar(50)
        , Phone varchar(50)
    )
    """
cursor.execute(create_table_query)

In [8]:
# Insert Data into the table
for i in range(len(df)):
    sql = "INSERT INTO external_data.temples (Temple, Status, Latitude, Longitude, Address, City, County, State, Postal_Code, Country, Phone) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    values = (df['Temple'].iloc[i], df['Status'].iloc[i], df['Latitude'].iloc[i], df['Longitude'].iloc[i], df['Address'].iloc[i], df['City'].iloc[i], df['County'].iloc[i], df['State'].iloc[i], df['Postal_Code'].iloc[i], df['Country'].iloc[i], df['Phone'].iloc[i])
        
    # Execute the INSERT statement
    cursor.execute(sql, values)

# Commit the changes and close the connection
cnx.commit()
cnx.close()